## Задача 1. Количество параллельных экспериментов

Сколько несовместных независимых экспериментов можно запустить одновременно, если за время эксперимента собираем 10000 наблюдений?

Если решаем запустить 10 экспериментов, то на каждый эксперимент можно выделить по 1000 наблюдений, размер групп будет равен 500.

Параметры экспериментов:

- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import random


### авторское решение

тк знаем стандартное отклонение распределения, размер эффекта, и предельные ошибки 1 и 2 рода, то можем оценить минимальный размер группы для получения эффекта выше как мде

In [21]:
# параметры эксперимента
total_size = 10000
mean_ = 100
std_ = 10
effect = 0.03
alpha = 0.05
beta = 0.1

# оценка размера группы через эффект и мде
def estimate_sample_size(effect, std, alpha, beta):
    
    # обратная функция распределения для альфы
    t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    
    # обратная функция распределения для беты
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    
    # сумма стандартных отклонений
    var = 2 * std ** 2
    
    # оценка минимального размера группы
    sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
    
    return sample_size


# оценим необходимый размер групп
sample_size = estimate_sample_size(effect * 100, 10, alpha, beta)
print(f'sample_size = {sample_size}')

# вычислим количество экспериментов
count_exp = total_size / (sample_size * 2)
print(f'count_exp = {count_exp:0.1f}')


def estimate_ci_bernoulli(p, n, alpha=0.05):
    """Доверительный интервал для Бернуллиевской случайной величины."""
    t = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    std_n = np.sqrt(p * (1 - p) / n)
    return p - t * std_n, p + t * std_n

# Проверим, что при 21 эксперимента ошибки контролируются на заданных уровнях, а при 22 экспериментах нет.

for count_exp in [21, 22]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in tqdm(range(10000)):
        a, b = np.random.normal(mean_, std_, (2, sample_size,))
        b_effect = b * (1 + effect)
        errors_aa.append(stats.ttest_ind(a, b).pvalue < alpha)
        errors_ab.append(stats.ttest_ind(a, b_effect).pvalue >= alpha)

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')


  6%|▌         | 558/10000 [00:00<00:03, 2789.13it/s]

sample_size = 233
count_exp = 21.5


  3%|▎         | 289/10000 [00:00<00:03, 2889.64it/s]

count_exp = 21
sample_size = 238
оценка вероятности ошибки I рода = 0.0517
  доверительный интервал = [0.0474, 0.0560]
оценка вероятности ошибки II рода = 0.1027
  доверительный интервал = [0.0968, 0.1086]


100%|██████████| 10000/10000 [00:03<00:00, 2918.79it/s]

count_exp = 22
sample_size = 227
оценка вероятности ошибки I рода = 0.0535
  доверительный интервал = [0.0491, 0.0579]
оценка вероятности ошибки II рода = 0.1160
  доверительный интервал = [0.1097, 0.1223]


In [ ]:
# sample_size = 233
# count_exp = 21.5

# count_exp = 21
# sample_size = 238
# оценка вероятности ошибки I рода = 0.0461
#   доверительный интервал = [0.0420, 0.0502]
# оценка вероятности ошибки II рода = 0.1018
#   доверительный интервал = [0.0959, 0.1077]

# count_exp = 22
# sample_size = 227
# оценка вероятности ошибки I рода = 0.0485
#   доверительный интервал = [0.0443, 0.0527]
# оценка вероятности ошибки II рода = 0.1165
#   доверительный интервал = [0.1102, 0.1228]

### мое решение

In [6]:
# параметры нормального распределения
mean = 100 
std_dev = 10

# всего наблюдений
N = 10000 
alpha = 0.05
beta = 0.1
uplift = 1.03


In [39]:
def create_groups(groups_cnt):
    single_size = int(N / groups_cnt)
    
    exp_sample = np.random.normal(loc=mean, scale=std_dev, size=single_size)
    random.shuffle(exp_sample)
    
    total_users = len(exp_sample)
    half_size = total_users // 2
    
    test_distr = exp_sample[:half_size]
    control_distr = exp_sample[half_size:]
    
#     print(len(test_distr), round(test_distr.mean(), 2))
#     print(len(control_distr), round(control_distr.mean(), 2))
    _, aa_pvalue = stats.ttest_ind(control_distr, test_distr)
    
    test_distr = test_distr * uplift
#     print(len(test_distr), round(test_distr.mean(), 2))
#     print(len(control_distr), round(control_distr.mean(), 2))
#     print(round(test_distr.mean() / control_distr.mean(), 2))
    _, ab_pvalue = stats.ttest_ind(control_distr, test_distr)

    return aa_pvalue, ab_pvalue


In [55]:
for count_sample in range(2, 30):
    
    aa_pvalues = []
    ab_pvalues = []
    
    for _ in range(10000):
        aa_pvalue, ab_pvalue = create_groups(count_sample)
        aa_pvalues.append(aa_pvalue)
        ab_pvalues.append(ab_pvalue)
        
    # ошибка первого рода = доля прокрашенных аа тестов
    first_type_sample = round(sum([1 if pval_aa < alpha else 0 for pval_aa in aa_pvalues]) / len(aa_pvalues), 5)
    
    # ошибка второго рода = доля непрокрашенных аб-тестов
    second_type_sample = round(1 - sum([1 if pval_ab < alpha else 0 for pval_ab in ab_pvalues]) / len(ab_pvalues), 5)
    
    print(count_sample, alpha, first_type_sample, beta, second_type_sample)
#     if first_type_sample > alpha or second_type_sample > beta:
#         print(count_sample, alpha, first_type_sample, beta, second_type_sample)
#         break
    

2 0.05 0.0479 0.1 0.0
3 0.05 0.0484 0.1 0.0
4 0.05 0.0498 0.1 0.0
5 0.05 0.052 0.1 0.0
6 0.05 0.0516 0.1 0.0
7 0.05 0.0507 0.1 0.0002
8 0.05 0.0489 0.1 0.0006
9 0.05 0.0501 0.1 0.0011
10 0.05 0.0534 0.1 0.0034
11 0.05 0.0482 0.1 0.0065
12 0.05 0.0524 0.1 0.0123
13 0.05 0.0463 0.1 0.0156
14 0.05 0.0505 0.1 0.0227
15 0.05 0.0487 0.1 0.0303
16 0.05 0.0491 0.1 0.0426
17 0.05 0.0461 0.1 0.0522
18 0.05 0.0484 0.1 0.0639
19 0.05 0.0485 0.1 0.0741
20 0.05 0.0493 0.1 0.0897
21 0.05 0.0494 0.1 0.1026
22 0.05 0.052 0.1 0.1216
23 0.05 0.0477 0.1 0.1332
24 0.05 0.0509 0.1 0.1468
25 0.05 0.0541 0.1 0.1664
26 0.05 0.0506 0.1 0.1724
27 0.05 0.0479 0.1 0.1876
28 0.05 0.0493 0.1 0.2029
29 0.05 0.0509 0.1 0.2232


## Задача 2. Количество параллельных экспериментов — 2

Задача похожа на предыдущую, только теперь решение принимается не независимо для каждого эксперимента.
Например, у нас есть 5 текстов для маркетинговой рассылки, хотим проверить, какой эффективнее работает и работает ли вообще.
Алгоритм будет следующий:

1. Формируем непересекающиеся контрольные и экспериментальные группы для каждого из 5 вариантов.
2. Проводим параллельно 5 экспериментов.
3. С помощью метода Холма определяем, в каких экспериментах были статистически значимые отличия.
4. Если значимых отличий не обнаружено, то говорим, что эффекта нет, все варианты отклоняем.
5. Если значимые отличия обнаружены, то из вариантов со значимым эффектом выбираем вариант с наименьшим значением p-value, будем использовать его.

Будем считать, что совершена ошибка I рода, если найдены значимые отличия, когда на самом деле их не было ни в одном из вариантов.

Будем считать, что совершена ошибка II рода, если:
- либо не найдено значимых отличий, когда на самом деле они были;
- либо выбранный для дальнейшего использования вариант на самом деле был без эффекта, при этом были варианты с эффектом.

Параметры экспериментов:

- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Замечание: при оценке вероятности ошибки II рода нужно рассматривать худший сценарий, когда эффект есть только в одном из экспериментов. Чем в большем количестве экспериментов будет эффект, тем меньше будет вероятность ошибки II рода.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.

### мое решение

In [3]:
total_size = 10000
mean_ = 100
std_ = 10
effect = 0.03
alpha = 0.05
beta = 0.1

In [18]:
for count_exp in range(2, 25):
    
    errors_aa = []
    errors_ab = []
    
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in tqdm(range(10000)):
        # создаю словарь из count_exp экспов для замеров по аа-тестам
        active_aa_exps = {}
        
        # создаю словарь из count_exp экспов для замеров по аb-тестам
        active_ab_exps = {}
        
        # генерю тест и контроль для каждого экспа
        for exp_num in range(count_exp):
            
            # каждому номеру теста соответствует список с aa_pval, alpha_aa_upd
            active_aa_exps[exp_num] = []
            
            # каждому номеру теста соответствует список с ab_pval, alpha_ab_upd
            active_ab_exps[exp_num] = []
            
            # генерю тест и контроль для аа-теста данной итерации
            a, b = np.random.normal(mean_, std_, (2, sample_size,))
            
            # считаю pvalue для аа-теста данной итерации
            _, aa_pval = stats.ttest_ind(a, b)
            active_aa_exps[exp_num].append(aa_pval)
            
            # для первого теста из count_exp добавляем эффект для замеров ошибки 2 рода
            if exp_num == 0:
                b = b * (1 + effect)
                active_ab_exps[exp_num].append('has_real_effect')
            else:
                active_ab_exps[exp_num].append('no_real_effect')
                
            # считаю pvalue для аb-теста данной итерации
            _, ab_pval = stats.ttest_ind(a, b)
            active_ab_exps[exp_num].append(ab_pval)
            
            
            
        # ранжирую по pvalue для поиска ошибок 1 рода для этой итерации
        active_aa_exps = dict(sorted(active_aa_exps.items(), key=lambda item: item[1]))
        index = 1
        for key in active_aa_exps:
            # cохраняю индивидуальные pvalue для поиска ошибок 1 рода для этой итерации
            local_alpha = alpha / (count_exp + 1 - index)
            active_aa_exps[key].append(local_alpha)
            index += 1 

        # складываю результат поиска ошибок 1 рода для этой итерации
        # если хотя бы в 1 тесте из списка доступных найден эффект - кладем 1 в errors_aa
        # если в списке доступных нигде не найден эффект - кладем 0 в errors_aa
        available_tests_list = list(active_aa_exps.keys()).copy()
        found_error_1_type = 0
        for i in range(len(available_tests_list)):
            # если данный скорректированный pvalue больше полученного для аа-теста, возводим флаг и прерываемся
            if active_aa_exps[available_tests_list[i]][1] > active_aa_exps[available_tests_list[i]][0]:
                found_error_1_type = 1
                break
        errors_aa.append(found_error_1_type)

        
        # ранжирую по pvalue для поиска ошибок 2 рода для этой итерации
        active_ab_exps = dict(sorted(active_ab_exps.items(), key=lambda item: item[1]))
        index = 1
        for key in active_ab_exps:
            # cохраняю индивидуальные pvalue для поиска ошибок 2 рода для этой итерации
            local_alpha = alpha / (count_exp + 1 - index)
            active_ab_exps[key].append(local_alpha)
            index += 1 

            
        # складываю результат поиска ошибок 2 рода для этой итерации
        # если в тесте с эффектом скорректированный pvalue меньше полученного для аb-теста, кладем 1 в errors_ab
        # если найден эффект в тесте без эффекта, кладем 1 в errors_ab
        # в противном случае кладем 0 в errors_ab
        available_tests_list = list(active_ab_exps.keys()).copy()
        found_error_2_type = 0
        for i in range(len(available_tests_list)):
            # если в тесте с эффектом 
            if active_ab_exps[available_tests_list[i]][1] == 'has_real_effect':
                # скорректированный pvalue меньше полученного для аb-теста
                if active_ab_exps[available_tests_list[i]][2] <= active_ab_exps[available_tests_list[i]][1]:
                    # возводим флаг и прерываемся
                    found_error_2_type = 1
                    break
                
            # если в тесте без эффекта
            elif active_ab_exps[available_tests_list[i]][1] == 'no_real_effect':
                # пограничное скорректированное значение pvalue больше полученного pvalue
                if active_ab_exps[available_tests_list[i]][2] > active_ab_exps[available_tests_list[i]][1]:
                    # возводим флаг и прерываемся
                    found_error_2_type = 1
                    break
        errors_ab.append(found_error_2_type)

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    
    print('count_exp =', count_exp)
    print(estimated_first_type_error, len(errors_aa))
    print(estimated_second_type_error, len(errors_ab), end='\n\n\n')

  1%|          | 77/10000 [00:00<00:13, 761.75it/s]

count_exp = 2
0.048 10000
0.0 10000




  1%|          | 58/10000 [00:00<00:17, 579.38it/s]

count_exp = 3
0.0512 10000
0.0 10000




  0%|          | 50/10000 [00:00<00:20, 491.58it/s]

...

count_exp = 22
0.0483 10000
0.0 10000




  0%|          | 13/10000 [00:00<01:18, 126.66it/s]

count_exp = 23
0.0466 10000
0.0 10000




100%|██████████| 10000/10000 [01:23<00:00, 120.35it/s]

count_exp = 24
0.0498 10000
0.0 10000




### авторское решение

In [22]:
def method_holm(pvalues, alpha=0.05):
    """Применяет метод Холма для проверки значимости изменений.
    
    pvalues - List[float] - список pvalue.
    alpha - float, уровень значимости.
    return - np.array, массив из нулей и единиц, 0 - эффекта нет, 1 - эффект есть.
    """
    m = len(pvalues)
    array_alpha = np.arange(m, 0, -1)
    array_alpha = alpha / array_alpha
    sorted_pvalue_indexes = np.argsort(pvalues)
    res = np.zeros(m)
    for idx, pvalue_index in enumerate(sorted_pvalue_indexes):
        pvalue = pvalues[pvalue_index]
        alpha_ = array_alpha[idx]
        if pvalue < alpha_:
            res[pvalue_index] = 1
        else:
            break
    res = res.astype(int)
    return res

# Проверим, что при 12 эксперимента ошибки контролируются на заданных уровнях, а при 13 экспериментах нет.

for count_exp in [12, 13]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in tqdm(range(10000)):
        list_ab_values = [
            np.random.normal(mean_, std_, (2, sample_size))
            for _ in range(count_exp)
        ]
        # синтетический А/А тест
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        aa_with_effect = method_holm(pvalues, alpha)
        errors_aa.append(np.sum(aa_with_effect) > 0)

        # Синтетический А/Б тест.
        # Достаточно проверить случай, когда эффект есть лишь в одном из экспериментов,
        # так как при наличии эффектов в большем кол-ве экспериментов ошибок II рода станет меньше.
        # Добавим эффект в первый эксперимент (не важно в какой добавлять, так как данные случайные)
        list_ab_values[0][1] *= 1 + effect
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        ab_with_effect = method_holm(pvalues, alpha)
        if np.sum(ab_with_effect) == 0:
            # если эффектов не найдено, то это ошибка
            errors_ab.append(True)
        else:
            # если эффектов найден где его нет, то это ошибка
            errors_ab.append(np.min(pvalues) != pvalues[0])

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')

  0%|          | 23/10000 [00:00<00:43, 226.90it/s]

count_exp = 12
sample_size = 416
оценка вероятности ошибки I рода = 0.0509
  доверительный интервал = [0.0466, 0.0552]
оценка вероятности ошибки II рода = 0.0866
  доверительный интервал = [0.0811, 0.0921]


100%|██████████| 10000/10000 [00:45<00:00, 218.95it/s]

count_exp = 13
sample_size = 384
оценка вероятности ошибки I рода = 0.0499
  доверительный интервал = [0.0456, 0.0542]
оценка вероятности ошибки II рода = 0.1202
  доверительный интервал = [0.1138, 0.1266]


Получили, что при 12 экспериментах всё корректно, а при 13 экспериментах вероятность ошибки II рода больше 0.1.

Получается, если мы проводим множественное тестирование, то за раз можем проверить меньшее количество гипотез, чем если бы проверяли независимые гипотезы. Это логично, так как мы предъявляем более строгое требование к вероятностям ошибок первого рода при множественном тестировании. Из-за этого наш критерий дополнительно перестраховывается и реже говорит, что эффект есть, что приводит к увеличению вероятности ошибок II рода. Компенсировать увеличение вероятности ошибок II рода приходится за счёт увеличения размера групп.